This notebook demonstrates the global optimization of a 2-layer AR coating for a 2 $\mu m$ silicon
solar cell and should reproduce the efficiency reported in Table III ($L = 2$) in "Accelerating the discovery of multi-layer nanostructures with analytic differentiation of the transfer matrix equations" by J. F. Varner, D. Wert, A. Matari, R. Nofal, and J. J. Foley IV.

In [21]:
from wptherml.wpml import multilayer
from wptherml.datalib import datalib
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import basinhopping
import numpy as np
import time

### Define a 2 micron silicon slab with a SiO2 and AlN coating...
### we will optimize over the SiO2 and AlN coatings only.
structure = {

        'Material_List' : ['Air', 'AlN','W','Air'],
        ### Thicknesses just chosen arbitrarily, replace with "optimal" values
        'Thickness_List': [0, 70e-9, 900e-9, 0],
        ### add a number to Gradient_List to optimize over more layers
        'Gradient_List': [1],
        'Lambda_List': [300e-9, 6000e-9, 1000],
        'STPV_EMIT': 1,
        'Temperature': 1300
        }

### create instance of the multilayer
cc = multilayer(structure)
print(cc.spectral_efficiency_val)
length = len(cc.conversion_efficiency_grad)

### given an array of thicknesses of the AR layers, update
### the structure and compute its conversion efficiency
def update_multilayer(x):
    for i in range(0,len(x)):
        cc.d[i+1] = x[i]*1e-9
    ### now we have the new structure, update fresnel quantities
    cc.fresnel()
    cc.thermal_emission()
    ### now we have new emissivity, update thermal emission
    cc.stpv_se()

    ### return negative of luminous efficiency
    return -cc.spectral_efficiency_val*100

### given an array of thicknesses of the AR layers, update
### the structure and compute the gradient vector of conversion efficiency wrt AR layer thicknesses
def analytic_grad(x0):
    cur = update_multilayer(x0)
    cc.fresnel_prime()
    cc.stpv_se_grad()
    g = cc.spectral_efficiency_grad
    ### return the negative of the gradient scaled by 10^-9 
    ### will make step-size more reasonable
    return -g*1e-9

### This is a finite-difference version of the gradient
### which may be called by SuperFunc if one wants to compare it
### to analytic_grad
def BuildGradient(x0):
    dim = len(x0)
    h0 = 0.1*np.ones(dim)
    g = np.zeros(dim)
    for i in range(0,dim):
        xpass = np.copy(x0)
        fx = x0[i] + h0[i]
        bx = x0[i] - h0[i]
        xpass[i] = fx
        efx = update_multilayer(xpass)
        xpass[i] = bx
        ebx = update_multilayer(xpass)

        run = 2*h0[i]
        g[i] = (efx-ebx)/run
    return g

### function that gets efficiency and gradient given an array 
### of AR layer thicknesses
def SuperFunc(x):
    en = update_multilayer(x)
    gr = analytic_grad(x)
    return en, gr



0.5089608224059269


In [23]:
x = [100]
print(update_multilayer(x))
print(analytic_grad(x))
print(BuildGradient(x))

-52.775486540135795
[-0.00061931]
[-0.31037693]


In [ ]:
### prints efficiency and time
def print_fun(x, f, accepted):
    c_time = time.time()
    print(f,",",c_time)

### called by the basin hopping algorithm to initiate new
### local optimizations
def my_take_step(x):
    xnew = np.copy(x)
    dim = len(xnew)
    for i in range(0,dim):
        rn = 50.*np.abs(np.random.randn())
        xnew[i] = rn
    return xnew

### bounds on basin hopping solutions
class MyBounds(object):
      ### note xmax and xmin need to have as many elements as there are thicknesses that are varied
    def __init__(self, xmax=49.01*np.ones(length), xmin=0.01*np.ones(length)):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

# the bounds for L-BFGS-B updates!
bfgs_xmin = 0.01*np.ones(length)
bfgs_xmax = 49.01*np.ones(length)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

### arguments for basin hopping algorithm
minimizer_kwargs = {"method": "L-BFGS-B", "jac": True, "bounds": bfgs_bounds}
mybounds = MyBounds()

### initial guess for AR layer thicknesses!
xs = np.ones(length)*20.

### call basin hopping!
ret = basinhopping(SuperFunc, xs, minimizer_kwargs=minimizer_kwargs, niter=100, take_step=my_take_step, callback=print_fun, accept_test=mybounds)

### print optimimal result!
print(ret.x)
print(update_multilayer(ret.x))
